In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import tqdm
import os
import wandb

In [36]:
wandb.login()

#wandb.init(project="amazon-books-reviews-sentiment", name="T6-6class", reinit=True)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [37]:
# Hyperparameters
mb_size = 64 # batch size
Z_dim = 1000 # Random noise size
h_dim = 128 # Hiden layer Size
c_dim = 10 # class 
X_dim = 784  # 28 x 28: Input size
lr = 1e-4 # Learning rate

In [38]:
# Load MNIST data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the 28x28 image to 784
])

train_dataset = datasets.MNIST(root='../MNIST', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=mb_size, shuffle=True)



In [39]:
# Xavier Initialization: A good way to initialize weights to make the gradient have the same magnitude in all layers
def xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


In [40]:
# To onehot
def to_onehot(x, num_classes=10):
    if isinstance(x, int):
        x = torch.tensor([x], dtype=torch.long)
    elif not isinstance(x, torch.Tensor):
        raise TypeError("Input must be an int or a torch.Tensor")

    x = x.long()  # ensure it's LongTensor for one-hot indexing
    one_hot = torch.zeros(x.size(0), num_classes, device=x.device)
    one_hot.scatter_(1, x.view(-1, 1), 1)
    return one_hot

In [41]:
# Generator
class Generator(nn.Module):
    def __init__(self, z_dim, h_dim, x_dim, c_dim):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(z_dim + c_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, x_dim)
        self.apply(xavier_init)

    def forward(self, z,c):
        v = torch.cat((z, c), 1)
        h = F.relu(self.fc1(v))
        out = torch.sigmoid(self.fc2(h))
        return out


In [42]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, x_dim, h_dim, c_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(x_dim + c_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, 1)
        self.apply(xavier_init)

    def forward(self, x, c):
        v = torch.cat((x, c), 1)
        h = F.relu(self.fc1(v))
        out = torch.sigmoid(self.fc2(h))
        return out


In [43]:
# Training
def cGANTraining(G, D, loss_fn, train_loader):
    G.train()
    D.train()

    D_loss_real_total = 0
    D_loss_fake_total = 0
    G_loss_total = 0
    t = tqdm.tqdm(train_loader)
    
    for it, (X_real, labels) in enumerate(t):
        # Prepare real data
        X_real = X_real.float().to(device)
        c = to_onehot(labels).to(device)
        
        # Sample noise and labels
        z = torch.randn(X_real.size(0), Z_dim).to(device)
        ones_label = torch.ones(X_real.size(0), 1).to(device)
        zeros_label = torch.zeros(X_real.size(0), 1).to(device)

        # ================= Train Discriminator =================
        G_sample = G(z,c) # generete an image
        D_real = D(X_real, c) # Identify real image
        D_fake = D(G_sample.detach(),c) # Identify fake image

        D_loss_real = loss_fn(D_real, ones_label)
        D_loss_fake = loss_fn(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake
        D_loss_real_total += D_loss_real.item()
        D_loss_fake_total += D_loss_fake.item()

        # train only discriminator
        D_solver.zero_grad()
        D_loss.backward()
        D_solver.step()

        # ================= Train Generator ====================
        z = torch.randn(X_real.size(0), Z_dim).to(device) # construct new random noise
        G_sample = G(z,c) # generate a image
        D_fake = D(G_sample,c) #Identify fake image

        G_loss = loss_fn(D_fake, ones_label) #how well did the generetor fool the discriminator
        G_loss_total += G_loss.item()

        #Train only the generator
        G_solver.zero_grad()
        G_loss.backward()
        G_solver.step()

    # ================= Logging =================
    D_loss_real_avg = D_loss_real_total / len(train_loader)
    D_loss_fake_avg = D_loss_fake_total / len(train_loader)
    D_loss_avg = D_loss_real_avg + D_loss_fake_avg
    G_loss_avg = G_loss_total / len(train_loader)
    
    wandb.log({
        "D_loss_real": D_loss_real_avg,
        "D_loss_fake": D_loss_fake_avg,
        "D_loss": D_loss_avg,
        "G_loss": G_loss_avg,
        "batch_size": mb_size,
        "Z_dim": Z_dim,
        "X_dim": X_dim,
        "h_dim": h_dim,
        "lr": lr,
    })

    return G, D, G_loss_avg, D_loss_avg


In [44]:
# Save a image with 0 to 9 generated from that batch
def save_sample(G, epoch, mb_size, Z_dim):
    out_dir = "out_vanila_GAN2"
    G.eval()
    with torch.no_grad():
        num_classes = 10
        z = torch.randn(num_classes, Z_dim).to(device)

        # One-hot encoded class conditions
        c = torch.zeros(num_classes, num_classes).to(device)
        c[torch.arange(num_classes), torch.arange(num_classes)] = 1

        samples = G(z, c).detach().cpu().numpy()

    fig = plt.figure(figsize=(10, 1))
    for i in range(num_classes):
        plt.subplot(1, num_classes, i + 1)
        plt.axis('off')
        plt.imshow(samples[i].reshape(28, 28), cmap='Greys_r')

    os.makedirs(out_dir, exist_ok=True)
    plt.savefig(f'{out_dir}/{str(epoch).zfill(3)}.png', bbox_inches='tight')
    plt.close(fig)

In [45]:
########################### Main #######################################
wandb_log = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Instantiate models
G = Generator(Z_dim, h_dim, X_dim, c_dim).to(device)
D = Discriminator(X_dim, h_dim, c_dim).to(device)

# Optimizers
G_solver = optim.Adam(G.parameters(), lr=lr)
D_solver = optim.Adam(D.parameters(), lr=lr)

# Loss function
def my_bce_loss(preds, targets):
    return F.binary_cross_entropy(preds, targets)

#loss_fn = nn.BCEWithLogitsLoss()
loss_fn = my_bce_loss

if wandb_log: 
    wandb.init(project="conditional-gan-mnist")

    # Log hyperparameters
    #wandb.config.update({
   # #    "batch_size": mb_size,
    #    "Z_dim": Z_dim,
    #    "X_dim": X_dim,
    #    "h_dim": h_dim,
   # #    "lr": lr,
    #})



#Train epochs
epochs = 1000

for epoch in range(epochs):
    G, D, G_loss_avg, D_loss_avg= cGANTraining(G, D, loss_fn, train_loader)

    print(f'epoch{epoch}; D_loss: {D_loss_avg:.4f}; G_loss: {G_loss_avg:.4f}')

    #if G_loss_avg < best_g_loss:
    #    best_g_loss = G_loss_avg
    #    torch.save(G.state_dict(), os.path.join(save_dir, 'G_best.pth'))
    #    torch.save(D.state_dict(), os.path.join(save_dir, 'D_best.pth'))
    #    print(f"Saved Best Models at epoch {epoch} | G_loss: {best_g_loss:.4f}")

    save_sample(G, epoch, mb_size, Z_dim)




D_loss,▇█▆▁▁
D_loss_fake,▄██▂▁
D_loss_real,█▆▃▁▁
G_loss,▄▁▁▆█
X_dim,▁▁▁▁▁
Z_dim,▁▁▁▁▁
batch_size,▁▁▁▁▁
h_dim,▁▁▁▁▁
lr,▁▁▁▁▁
D_loss,0.10331
D_loss_fake,0.04522


100%|██████████| 938/938 [00:09<00:00, 94.56it/s] 


epoch0; D_loss: 0.2973; G_loss: 2.8422


100%|██████████| 938/938 [00:10<00:00, 89.33it/s]


epoch1; D_loss: 0.3153; G_loss: 2.0262


...

epoch997; D_loss: 0.7486; G_loss: 1.9481


100%|██████████| 938/938 [00:10<00:00, 93.32it/s]


epoch998; D_loss: 0.7521; G_loss: 1.9493


100%|██████████| 938/938 [00:10<00:00, 89.89it/s]


epoch999; D_loss: 0.7513; G_loss: 1.9522
